In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os
import nept
import scipy

from loading_data import get_data, extract_xy, median_filter
from utils_maze import get_trials

In [ ]:
thisdir = os.getcwd()
dataloc = os.path.join(thisdir, 'cache', 'data')
pickle_filepath = os.path.join(thisdir, "cache", "pickled")
output_filepath = os.path.join(thisdir, "plots", "correcting_position")

In [ ]:
import info.r066d1 as r066d1
info = r066d1

In [ ]:
events = nept.load_events(os.path.join(dataloc, info.event_filename), info.event_labels)

In [ ]:
# Load raw position from file
filename = os.path.join(dataloc, info.position_filename)
nvt_data = nept.load_nvt(filename)
targets = nvt_data['targets']
times = nvt_data['time']

In [ ]:
# Initialize x, y arrays
x = np.zeros(targets.shape)
y = np.zeros(targets.shape)

# X and Y are stored in a custom bitfield. See Neuralynx data file format documentation for details.
# Briefly, each record contains up to 50 targets, each stored in 32bit field.
# X field at [20:31] and Y at [4:15].
# TODO: make into a separate function in nept
for target in range(targets.shape[1]):
    this_sample = targets[:, target]
    for sample in range(targets.shape[0]):
        # When the bitfield is equal to zero there is no valid data for that field
        # and remains zero for the rest of the bitfields in the record.
        if this_sample[target] == 0:
            break
        x[sample, target], y[sample, target] = extract_xy(int(this_sample[sample]))

# Replacing targets with no samples with nan instead of 0
x[x == 0] = np.nan
y[y == 0] = np.nan

# Scale the positions
x /= info.scale_targets
y /= info.scale_targets

In [ ]:
plt.plot(x, y, "b.")
plt.show()

In [ ]:
plt.plot(times, y, "b.")
plt.show()

In [ ]:
x[9220:9230]

In [ ]:
# led_padding = 0.0
dist_to_feeder = 20

# Find times when feeder led is on
ledon = sorted(np.append(events['led1'], events['led2'], 0))
ledoff = events["ledoff"]

led_idx = np.zeros(times.shape, dtype=bool)

for start, stop in zip(ledon, ledoff):
    led_idx[np.logical_and(times>=start, times<=stop)] = True

# Find times when target is close to a feeder location
x_idx = np.logical_or(np.abs(x - info.path_pts['feeder1'][0]) <= dist_to_feeder,
                      np.abs(x - info.path_pts['feeder2'][0]) <= dist_to_feeder)

y_idx = np.logical_or(np.abs(y - info.path_pts['feeder1'][1]) <= dist_to_feeder,
                      np.abs(y - info.path_pts['feeder2'][1]) <= dist_to_feeder)

xy_idx = np.logical_or(x_idx, y_idx)

# Find times to remove when led is on and target is close to a feeder location
remove_idx = np.logical_and(led_idx[..., np.newaxis], xy_idx)

# Remove idx when led is on and target is close to feeder location
x[remove_idx] = np.nan
y[remove_idx] = np.nan

In [ ]:
remove_idx.shape

In [ ]:
# # Removing the contaminated samples that are closest to the feeder location
# def remove_feeder_contamination(targets, current_feeder, dist_to_feeder=20):
#     targets = np.array(targets)
    
#     remove_idx = np.abs(targets - current_feeder[:, np.newaxis]) < dist_to_feeder
#     targets[remove_idx] = np.nan
    
#     return targets

In [ ]:
np.where(led_idx==True)

In [ ]:
x[87884:87890]

In [ ]:
remove_idx[9041:9048]

In [ ]:
led_padding = 0.03
# Finding which feeder led is on over time
leds = []
leds.extend([(event, 'led2') for event in events['led1']])
leds.extend([(event, 'led1') for event in events['led2']])
sorted_leds = sorted(leds)

ledoff = events["ledoff"]

# Get an array of feeder locations when that feeder is actively flashing
feeder_x_location = np.empty(times.shape[0]) * np.nan
feeder_y_location = np.empty(times.shape[0]) * np.nan

for (start, label), stop in zip(sorted_leds, ledoff):
    x_location = info.path_pts['feeder1'][0] if label == 'led1' else info.path_pts['feeder2'][0]
    y_location = info.path_pts['feeder1'][1] if label == 'led1' else info.path_pts['feeder2'][1]
        
    idx = np.where(np.logical_and(times>=start, times<=stop+led_padding))[0]
    feeder_x_location[idx] = x_location
    feeder_y_location[idx] = y_location

In [ ]:
# Removing the contaminated samples that are closest to the feeder location
def remove_feeder_contamination(targets, current_feeder, dist_to_feeder=20):
    targets = np.array(targets)
    
    remove_idx = np.abs(targets - current_feeder[:, np.newaxis]) < dist_to_feeder
    targets[remove_idx] = np.nan
    
    return targets
    
x = remove_feeder_contamination(x, feeder_x_location)
y = remove_feeder_contamination(y, feeder_y_location)

In [ ]:
# Removing the problem samples that are furthest from the previous location
def remove_based_on_std(original_targets, std_thresh=2):
    targets = np.array(original_targets)
    stds = np.nanstd(targets, axis=1)[:, np.newaxis]
    
    # find idx where there is a large variation between targets
    problem_samples = np.where(stds > std_thresh)[0]
    
    for i in problem_samples:
        # find the previous mean to help determine which target is an issue
        previous_idx = i-1
        previous_mean = np.nanmean(targets[previous_idx])
        
        # if previous sample is nan, compare current sample to the one before that
        while np.isnan(previous_mean):
            previous_idx -= 1
            previous_mean = np.nanmean(targets[previous_idx])
        
        # remove problem target
        idx = np.nanargmax(np.abs(targets[i] - previous_mean))
        targets[i][idx] = np.nan
    
    return targets

# x = remove_based_on_std(x)
# y = remove_based_on_std(y)

# Calculating the mean of the remaining targets
x = np.nanmean(x, axis=1)
y = np.nanmean(y, axis=1)

xx = np.array(x)
yy = np.array(y)
ttimes = np.array(times)


# Interpolate positions to replace nans during maze phases
def interpolate(time, array, nan_idx):
    f = scipy.interpolate.interp1d(time[~nan_idx], array[~nan_idx], 
                                   kind='linear', bounds_error=False)
    array[nan_idx] = f(time[nan_idx])

maze_phases = ["phase1", "phase2", "phase3"]
for task_time in info.task_times.keys():
    if task_time in maze_phases:
        trial_epochs = get_trials(events, info.task_times[task_time])
        for start, stop in zip(trial_epochs.starts, trial_epochs.stops):
            idx = (times >= start) & (times < stop)
            
            this_x = x[idx]
            this_y = y[idx]
            this_times = times[idx]

            # Finding nan idx
            x_nan_idx = np.isnan(this_x)
            y_nan_idx = np.isnan(this_y)
            nan_idx = x_nan_idx | y_nan_idx

            interpolate(this_times, this_x, nan_idx)
            interpolate(this_times, this_y, nan_idx)

            xx[idx] = this_x
            yy[idx] = this_y

# Finding nan idx
x_nan_idx = np.isnan(xx)
y_nan_idx = np.isnan(yy)
nan_idx = x_nan_idx | y_nan_idx

# Removing nan idx
xx = xx[~nan_idx]
yy = yy[~nan_idx]
ttimes = ttimes[~nan_idx]

# Apply a median filter
xx, yy = median_filter(xx, yy, kernel=11)
s
position = nept.Position(np.hstack(np.array([xx, yy])[..., np.newaxis]), ttimes)

plt.plot(position.x, position.y, "k.", ms=2)
plt.show()

In [ ]:
# Determining the percent of samples used
len(xx) / len(targets) * 100

In [ ]:
plt.plot(position.time, position.y, "k.", ms=2)
plt.show()

In [ ]:
def plot_trials(info, position, events, filepath):
    for phase in ["phase1", "phase2", "phase3"]:
        trial_epochs = get_trials(events, info.task_times[phase])
        for trial_idx in range(trial_epochs.n_epochs):
            start = trial_epochs[trial_idx].start
            stop = trial_epochs[trial_idx].stop

            trial = position.time_slice(start, stop)
            plt.plot(trial.x, trial.y, "k.")
            title = info.session_id + " " + phase + " trial" + str(trial_idx)
            plt.title(title)
            if filepath is not None:
                plt.savefig(os.path.join(filepath, title))
                plt.close()
            else:
                plt.show()
                plt.close()

In [ ]:
plot_trials(info, position, events, filepath=None)